In [1]:
from numpy import array
import tensorflow as tf
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [32]:
def avg_relative_error(y, p):
    error = 0
    er = []
    for i in range(len(y)):
        if y[i] != 0 and p[i] != 0:
            e = (((p[i] - y[i]) / max(y[i], p[i])))
            er.append(e[0])
            error += e[0]
    error = error / len(y)
    return abs(error)

In [33]:
countries = ['US','Australia','Austria', 'Brazil', 'China', 'Czechia', 'Denmark', 'France', 'Germany', 'India', 'Italy', 'Korea_South', 'South_Africa', 'Spain', 'UK']
results = []
for country in countries:
    result = {'country':'', 'conf': 0, 'dead': 0, 'recov': 0}
    result['country'] = country
    file = '../Data_Preprocessing/Clean/' + country + '.csv'
    df = pd.read_csv(file)
    confirmed = df['Confirmed_Cases'].tolist()
    recoveries = df['Recoveries'].tolist()
    deaths = df['Deaths'].tolist()

    n_steps = 4

    X, y = split_sequence(confirmed, n_steps)
    X_r, y_r = split_sequence(recoveries, n_steps)
    X_d, y_d = split_sequence(deaths, n_steps)

    n_features = 1
    n_seq = 2
    n_steps = 2
    X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))
    X_r = X_r.reshape((X_r.shape[0], n_seq, 1, n_steps, n_features))
    X_d = X_d.reshape((X_d.shape[0], n_seq, 1, n_steps, n_features))
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features), return_sequences=True))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(1,1), activation='relu', return_sequences=True))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ConvLSTM2D(filters=32, kernel_size=(1,1), activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X, y, epochs=500, verbose=0)
    X_test, y_test = split_sequence(confirmed[25:48], 4)
    X_test = X_test.reshape((X_test.shape[0], n_seq, 1, n_steps, n_features))
    res = model.predict(X_test)
    result['conf'] = avg_relative_error(y_test, res)
    
    model.fit(X_r, y_r, epochs=500, verbose=0)
    
    X_test_r, y_test_r = split_sequence(recoveries[25:48], 4)
    X_test_r = X_test_r.reshape((X_test_r.shape[0], n_seq, 1, n_steps, n_features))
    res_r = model.predict(X_test_r)
    result['recov'] = avg_relative_error(y_test_r, res_r)
    
    model.fit(X_d, y_d, epochs=500, verbose=0)
    
    X_test_d, y_test_d = split_sequence(deaths[25:48], 4)
    X_test_d = X_test_d.reshape((X_test_d.shape[0], n_seq, 1, n_steps, n_features))
    res_d = model.predict(X_test_d)
    result['dead'] = avg_relative_error(y_test_d, res_d)
    print(result)
    results.append(result)

results

{'country': 'US', 'conf': 0.3314393678631045, 'dead': 0.27102164532008927, 'recov': 0.14752372315055445}
{'country': 'Australia', 'conf': 0.1890699212488375, 'dead': 0.12253479581130178, 'recov': 0.07400957847896375}
{'country': 'Austria', 'conf': 0.5366294533994637, 'dead': 0.0, 'recov': 0.017156737415414108}
{'country': 'Brazil', 'conf': 0.0009298089303468403, 'dead': 0.0, 'recov': 0.0}
{'country': 'China', 'conf': 0.3184692742008912, 'dead': 0.03898696885689309, 'recov': 0.026872522283443494}
{'country': 'Czechia', 'conf': 0.20927690753811284, 'dead': 0.0, 'recov': 0.0}
{'country': 'Denmark', 'conf': 0.24846672031440234, 'dead': 0.0, 'recov': 0.039904930089649404}
{'country': 'France', 'conf': 0.0710045852159199, 'dead': 0.22857704758644104, 'recov': 0.003716886043548584}
{'country': 'Germany', 'conf': 0.4574022889137268, 'dead': 0.02578266984538028, 'recov': 0.1847235121225056}
{'country': 'India', 'conf': 0.16975452868561997, 'dead': 0.0, 'recov': 0.0}
{'country': 'Italy', 'conf':

[{'country': 'US',
  'conf': 0.3314393678631045,
  'dead': 0.27102164532008927,
  'recov': 0.14752372315055445},
 {'country': 'Australia',
  'conf': 0.1890699212488375,
  'dead': 0.12253479581130178,
  'recov': 0.07400957847896375},
 {'country': 'Austria',
  'conf': 0.5366294533994637,
  'dead': 0.0,
  'recov': 0.017156737415414108},
 {'country': 'Brazil',
  'conf': 0.0009298089303468403,
  'dead': 0.0,
  'recov': 0.0},
 {'country': 'China',
  'conf': 0.3184692742008912,
  'dead': 0.03898696885689309,
  'recov': 0.026872522283443494},
 {'country': 'Czechia',
  'conf': 0.20927690753811284,
  'dead': 0.0,
  'recov': 0.0},
 {'country': 'Denmark',
  'conf': 0.24846672031440234,
  'dead': 0.0,
  'recov': 0.039904930089649404},
 {'country': 'France',
  'conf': 0.0710045852159199,
  'dead': 0.22857704758644104,
  'recov': 0.003716886043548584},
 {'country': 'Germany',
  'conf': 0.4574022889137268,
  'dead': 0.02578266984538028,
  'recov': 0.1847235121225056},
 {'country': 'India', 'conf': 0.1

In [30]:
file = '../Data_Preprocessing/Clean/' + 'South_Africa' + '.csv'
df = pd.read_csv(file)
confirmed = df['Confirmed_Cases'].tolist()
recoveries = df['Recoveries'].tolist()
deaths = df['Deaths'].tolist()

n_steps = 4

X, y = split_sequence(confirmed, n_steps)
X_r, y_r = split_sequence(recoveries, n_steps)
X_d, y_d = split_sequence(deaths, n_steps)

n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))
X_r = X_r.reshape((X_r.shape[0], n_seq, 1, n_steps, n_features))
X_d = X_d.reshape((X_d.shape[0], n_seq, 1, n_steps, n_features))
    
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features), return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(1,1), activation='relu', return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ConvLSTM2D(filters=32, kernel_size=(1,1), activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=500, verbose=1)
X_test, y_test = split_sequence(confirmed[25:48], 4)
X_test = X_test.reshape((X_test.shape[0], n_seq, 1, n_steps, n_features))
res = model.predict(X_test)
mean_squared_error(y_test, res)

Train on 77 samples
Epoch 1/500
77/77 [==============================] - 10s 131ms/sample - loss: 3105.0469
Epoch 2/500
77/77 [==============================] - 0s 3ms/sample - loss: 3073.5274
Epoch 3/500
77/77 [==============================] - 0s 3ms/sample - loss: 3057.9261
Epoch 4/500
77/77 [==============================] - 0s 3ms/sample - loss: 3003.1412
Epoch 5/500
77/77 [==============================] - 0s 3ms/sample - loss: 2996.7933
Epoch 6/500
77/77 [==============================] - 0s 3ms/sample - loss: 2901.5440
Epoch 7/500
77/77 [==============================] - 0s 3ms/sample - loss: 2856.0439
Epoch 8/500
77/77 [==============================] - 0s 3ms/sample - loss: 2778.0246
Epoch 9/500
77/77 [==============================] - 0s 3ms/sample - loss: 2727.2350
Epoch 10/500
77/77 [==============================] - 0s 3ms/sample - loss: 2709.7967
Epoch 11/500
77/77 [==============================] - 0s 3ms/sample - loss: 2612.1404
Epoch 12/500
77/77 [====================

Epoch 97/500
77/77 [==============================] - 0s 3ms/sample - loss: 269.5797
Epoch 98/500
77/77 [==============================] - 0s 3ms/sample - loss: 304.7298
Epoch 99/500
77/77 [==============================] - 0s 3ms/sample - loss: 130.5597
Epoch 100/500
77/77 [==============================] - 0s 3ms/sample - loss: 256.9201
Epoch 101/500
77/77 [==============================] - 0s 3ms/sample - loss: 145.2473
Epoch 102/500
77/77 [==============================] - 0s 3ms/sample - loss: 372.7024
Epoch 103/500
77/77 [==============================] - 0s 3ms/sample - loss: 264.3171
Epoch 104/500
77/77 [==============================] - 0s 3ms/sample - loss: 223.6399
Epoch 105/500
77/77 [==============================] - 0s 3ms/sample - loss: 198.7900
Epoch 106/500
77/77 [==============================] - 0s 3ms/sample - loss: 196.2124
Epoch 107/500
77/77 [==============================] - 0s 3ms/sample - loss: 151.7761
Epoch 108/500
77/77 [==============================] - 0s

77/77 [==============================] - 0s 3ms/sample - loss: 241.4516
Epoch 193/500
77/77 [==============================] - 0s 3ms/sample - loss: 83.3061
Epoch 194/500
77/77 [==============================] - 0s 3ms/sample - loss: 78.9428
Epoch 195/500
77/77 [==============================] - 0s 3ms/sample - loss: 123.7336
Epoch 196/500
77/77 [==============================] - 0s 3ms/sample - loss: 87.0586
Epoch 197/500
77/77 [==============================] - 0s 3ms/sample - loss: 82.0545
Epoch 198/500
77/77 [==============================] - 0s 3ms/sample - loss: 162.9159
Epoch 199/500
77/77 [==============================] - 0s 3ms/sample - loss: 128.3475
Epoch 200/500
77/77 [==============================] - 0s 3ms/sample - loss: 57.2019
Epoch 201/500
77/77 [==============================] - 0s 3ms/sample - loss: 75.3871
Epoch 202/500
77/77 [==============================] - 0s 3ms/sample - loss: 115.5343
Epoch 203/500
77/77 [==============================] - 0s 3ms/sample - los

Epoch 288/500
77/77 [==============================] - 0s 3ms/sample - loss: 81.6891
Epoch 289/500
77/77 [==============================] - 0s 3ms/sample - loss: 184.7629
Epoch 290/500
77/77 [==============================] - 0s 3ms/sample - loss: 160.8429
Epoch 291/500
77/77 [==============================] - 0s 3ms/sample - loss: 67.9261
Epoch 292/500
77/77 [==============================] - 0s 3ms/sample - loss: 118.8602
Epoch 293/500
77/77 [==============================] - 0s 3ms/sample - loss: 131.5726
Epoch 294/500
77/77 [==============================] - 0s 3ms/sample - loss: 163.3057
Epoch 295/500
77/77 [==============================] - 0s 3ms/sample - loss: 101.3748
Epoch 296/500
77/77 [==============================] - 0s 3ms/sample - loss: 93.4215
Epoch 297/500
77/77 [==============================] - 0s 3ms/sample - loss: 212.5171
Epoch 298/500
77/77 [==============================] - 0s 3ms/sample - loss: 82.2823
Epoch 299/500
77/77 [==============================] - 0s 

77/77 [==============================] - 0s 3ms/sample - loss: 146.7730
Epoch 384/500
77/77 [==============================] - 0s 3ms/sample - loss: 88.9424
Epoch 385/500
77/77 [==============================] - 0s 3ms/sample - loss: 177.3285
Epoch 386/500
77/77 [==============================] - 0s 3ms/sample - loss: 67.3889
Epoch 387/500
77/77 [==============================] - 0s 3ms/sample - loss: 88.0229
Epoch 388/500
77/77 [==============================] - 0s 3ms/sample - loss: 111.5124
Epoch 389/500
77/77 [==============================] - 0s 3ms/sample - loss: 70.0302
Epoch 390/500
77/77 [==============================] - 0s 3ms/sample - loss: 85.6844
Epoch 391/500
77/77 [==============================] - 0s 3ms/sample - loss: 51.2808
Epoch 392/500
77/77 [==============================] - 0s 3ms/sample - loss: 97.3908
Epoch 393/500
77/77 [==============================] - 0s 3ms/sample - loss: 120.5396
Epoch 394/500
77/77 [==============================] - 0s 3ms/sample - loss

77/77 [==============================] - 0s 3ms/sample - loss: 46.1228
Epoch 480/500
77/77 [==============================] - 0s 3ms/sample - loss: 88.7812
Epoch 481/500
77/77 [==============================] - 0s 3ms/sample - loss: 53.5363
Epoch 482/500
77/77 [==============================] - 0s 3ms/sample - loss: 91.3843
Epoch 483/500
77/77 [==============================] - 0s 3ms/sample - loss: 84.4187
Epoch 484/500
77/77 [==============================] - 0s 3ms/sample - loss: 54.1614
Epoch 485/500
77/77 [==============================] - 0s 3ms/sample - loss: 103.3130
Epoch 486/500
77/77 [==============================] - 0s 3ms/sample - loss: 67.1598
Epoch 487/500
77/77 [==============================] - 0s 3ms/sample - loss: 32.6256
Epoch 488/500
77/77 [==============================] - 0s 3ms/sample - loss: 101.6835
Epoch 489/500
77/77 [==============================] - 0s 3ms/sample - loss: 142.1528
Epoch 490/500
77/77 [==============================] - 0s 3ms/sample - loss:

0.14504120250153066

In [31]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_36 (ConvLSTM2D) (None, 2, 1, 1, 128)      132608    
_________________________________________________________________
batch_normalization_11 (Batc (None, 2, 1, 1, 128)      512       
_________________________________________________________________
conv_lst_m2d_37 (ConvLSTM2D) (None, 2, 1, 1, 64)       49408     
_________________________________________________________________
batch_normalization_12 (Batc (None, 2, 1, 1, 64)       256       
_________________________________________________________________
conv_lst_m2d_38 (ConvLSTM2D) (None, 1, 1, 32)          12416     
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 32)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 32)              

In [15]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=500, verbose=1)

Train on 77 samples
Epoch 1/500
77/77 [==============================] - 3s 42ms/sample - loss: 4735959.6883
Epoch 2/500
77/77 [==============================] - 0s 969us/sample - loss: 4432496.5162
Epoch 3/500
77/77 [==============================] - 0s 941us/sample - loss: 4146234.0974
Epoch 4/500
77/77 [==============================] - 0s 951us/sample - loss: 3983280.8117
Epoch 5/500
77/77 [==============================] - 0s 940us/sample - loss: 3682714.1266
Epoch 6/500
77/77 [==============================] - 0s 923us/sample - loss: 3304920.1818
Epoch 7/500
77/77 [==============================] - 0s 950us/sample - loss: 2998306.5162
Epoch 8/500
77/77 [==============================] - 0s 938us/sample - loss: 2597719.6688
Epoch 9/500
77/77 [==============================] - 0s 941us/sample - loss: 2181397.3734
Epoch 10/500
77/77 [==============================] - 0s 958us/sample - loss: 1818123.7573
Epoch 11/500
77/77 [==============================] - 0s 935us/sample - loss: 13

77/77 [==============================] - 0s 965us/sample - loss: 350842.9140
Epoch 92/500
77/77 [==============================] - 0s 938us/sample - loss: 271928.5398
Epoch 93/500
77/77 [==============================] - 0s 959us/sample - loss: 325896.4107
Epoch 94/500
77/77 [==============================] - 0s 949us/sample - loss: 336626.1337
Epoch 95/500
77/77 [==============================] - 0s 937us/sample - loss: 345740.5197
Epoch 96/500
77/77 [==============================] - 0s 940us/sample - loss: 331971.0248
Epoch 97/500
77/77 [==============================] - 0s 949us/sample - loss: 329993.3182
Epoch 98/500
77/77 [==============================] - 0s 927us/sample - loss: 338817.2157
Epoch 99/500
77/77 [==============================] - 0s 926us/sample - loss: 338236.8571
Epoch 100/500
77/77 [==============================] - 0s 965us/sample - loss: 338870.3709
Epoch 101/500
77/77 [==============================] - 0s 922us/sample - loss: 332701.2606
Epoch 102/500
77/77 [

77/77 [==============================] - 0s 1ms/sample - loss: 330670.4145
Epoch 182/500
77/77 [==============================] - 0s 940us/sample - loss: 328328.4359
Epoch 183/500
77/77 [==============================] - 0s 949us/sample - loss: 324330.7971
Epoch 184/500
77/77 [==============================] - 0s 991us/sample - loss: 317777.6510
Epoch 185/500
77/77 [==============================] - 0s 955us/sample - loss: 321777.9261
Epoch 186/500
77/77 [==============================] - 0s 940us/sample - loss: 319222.6124
Epoch 187/500
77/77 [==============================] - 0s 970us/sample - loss: 329062.5357
Epoch 188/500
77/77 [==============================] - 0s 929us/sample - loss: 316897.8393
Epoch 189/500
77/77 [==============================] - 0s 937us/sample - loss: 318366.7873
Epoch 190/500
77/77 [==============================] - 0s 988us/sample - loss: 316232.7294
Epoch 191/500
77/77 [==============================] - 0s 933us/sample - loss: 315885.2784
Epoch 192/500
7

KeyboardInterrupt: 

In [47]:
X_test, y_test = split_sequence(confirmed[25:48], 4)
X_test = X_test.reshape((X_test.shape[0], n_seq, 1, n_steps, n_features))
res = model.predict(X_test)
mean_squared_error(y_test, res)

628.3510212387673

In [40]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_r, y_r, epochs=500, verbose=1)

Train on 77 samples
Epoch 1/500
77/77 [==============================] - 3s 42ms/sample - loss: 1073685.4706
Epoch 2/500
77/77 [==============================] - 0s 1ms/sample - loss: 1030930.5682
Epoch 3/500
77/77 [==============================] - 0s 1ms/sample - loss: 1008850.6071
Epoch 4/500
77/77 [==============================] - 0s 1ms/sample - loss: 977911.0211
Epoch 5/500
77/77 [==============================] - 0s 1ms/sample - loss: 952655.9481
Epoch 6/500
77/77 [==============================] - 0s 1ms/sample - loss: 909876.2898
Epoch 7/500
77/77 [==============================] - 0s 1ms/sample - loss: 874160.5406
Epoch 8/500
77/77 [==============================] - 0s 1ms/sample - loss: 821148.5162
Epoch 9/500
77/77 [==============================] - 0s 1ms/sample - loss: 763694.5649
Epoch 10/500
77/77 [==============================] - 0s 1ms/sample - loss: 700249.6510
Epoch 11/500
77/77 [==============================] - 0s 1ms/sample - loss: 636467.3293
Epoch 12/500
77/7

77/77 [==============================] - 0s 1ms/sample - loss: 104649.4330
Epoch 188/500
77/77 [==============================] - 0s 1ms/sample - loss: 102497.9218
Epoch 189/500
77/77 [==============================] - 0s 1ms/sample - loss: 105577.7353
Epoch 190/500
77/77 [==============================] - 0s 1ms/sample - loss: 133770.2993
Epoch 191/500
77/77 [==============================] - 0s 1ms/sample - loss: 141515.8929
Epoch 192/500
77/77 [==============================] - 0s 1ms/sample - loss: 149952.7656
Epoch 193/500
77/77 [==============================] - 0s 1ms/sample - loss: 154865.1203
Epoch 194/500
77/77 [==============================] - 0s 1ms/sample - loss: 164626.6672
Epoch 195/500
77/77 [==============================] - 0s 1ms/sample - loss: 133579.8736
Epoch 196/500
77/77 [==============================] - 0s 1ms/sample - loss: 127088.7406
Epoch 197/500
77/77 [==============================] - 0s 1ms/sample - loss: 113285.2530
Epoch 198/500
77/77 [==============

77/77 [==============================] - 0s 1ms/sample - loss: 45058.8856
Epoch 373/500
77/77 [==============================] - 0s 1ms/sample - loss: 44777.5697
Epoch 374/500
77/77 [==============================] - 0s 1ms/sample - loss: 44576.3758
Epoch 375/500
77/77 [==============================] - 0s 1ms/sample - loss: 44311.2413
Epoch 376/500
77/77 [==============================] - 0s 1ms/sample - loss: 44064.5911
Epoch 377/500
77/77 [==============================] - 0s 1ms/sample - loss: 43827.1950
Epoch 378/500
77/77 [==============================] - 0s 1ms/sample - loss: 43555.9225
Epoch 379/500
77/77 [==============================] - 0s 1ms/sample - loss: 43410.9651
Epoch 380/500
77/77 [==============================] - 0s 1ms/sample - loss: 43011.3451
Epoch 381/500
77/77 [==============================] - 0s 1ms/sample - loss: 42917.8749
Epoch 382/500
77/77 [==============================] - 0s 1ms/sample - loss: 42577.7874
Epoch 383/500
77/77 [=========================

In [41]:
X_test_r, y_test_r = split_sequence(recoveries[25:48], 4)
X_test_r = X_test_r.reshape((X_test_r.shape[0], n_seq, 1, n_steps, n_features))
res_r = model.predict(X_test_r)
mean_squared_error(y_test_r, res_r)

7.325915214097329

In [42]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_d, y_d, epochs=500, verbose=1)

Train on 77 samples
Epoch 1/500
77/77 [==============================] - 3s 44ms/sample - loss: 149520244.3636
Epoch 2/500
77/77 [==============================] - 0s 1ms/sample - loss: 144026453.5065
Epoch 3/500
77/77 [==============================] - 0s 1ms/sample - loss: 135916004.7792
Epoch 4/500
77/77 [==============================] - 0s 1ms/sample - loss: 125499537.6623
Epoch 5/500
77/77 [==============================] - 0s 1ms/sample - loss: 108600145.5584
Epoch 6/500
77/77 [==============================] - 0s 1ms/sample - loss: 91583809.0390
Epoch 7/500
77/77 [==============================] - 0s 1ms/sample - loss: 70455689.6623
Epoch 8/500
77/77 [==============================] - 0s 1ms/sample - loss: 51684581.8701
Epoch 9/500
77/77 [==============================] - 0s 1ms/sample - loss: 27324012.6753
Epoch 10/500
77/77 [==============================] - 0s 1ms/sample - loss: 13075884.6623
Epoch 11/500
77/77 [==============================] - 0s 1ms/sample - loss: 5944628

77/77 [==============================] - 0s 1ms/sample - loss: 3879944.8815
Epoch 184/500
77/77 [==============================] - 0s 1ms/sample - loss: 3817781.2078
Epoch 185/500
77/77 [==============================] - 0s 1ms/sample - loss: 3751713.4221
Epoch 186/500
77/77 [==============================] - 0s 1ms/sample - loss: 4065129.7403
Epoch 187/500
77/77 [==============================] - 0s 1ms/sample - loss: 3778008.5519
Epoch 188/500
77/77 [==============================] - 0s 1ms/sample - loss: 3779479.2597
Epoch 189/500
77/77 [==============================] - 0s 1ms/sample - loss: 3754000.5325
Epoch 190/500
77/77 [==============================] - 0s 1ms/sample - loss: 3759154.0651
Epoch 191/500
77/77 [==============================] - 0s 1ms/sample - loss: 3741474.0519
Epoch 192/500
77/77 [==============================] - 0s 1ms/sample - loss: 3712141.6039
Epoch 193/500
77/77 [==============================] - 0s 1ms/sample - loss: 3770495.6753
Epoch 194/500
77/77 [===

77/77 [==============================] - 0s 1ms/sample - loss: 2557567.3149
Epoch 366/500
77/77 [==============================] - 0s 1ms/sample - loss: 2673310.1234
Epoch 367/500
77/77 [==============================] - 0s 1ms/sample - loss: 2712552.2175
Epoch 368/500
77/77 [==============================] - 0s 1ms/sample - loss: 2549755.1201
Epoch 369/500
77/77 [==============================] - 0s 1ms/sample - loss: 2598377.6818
Epoch 370/500
77/77 [==============================] - 0s 1ms/sample - loss: 2728964.4838
Epoch 371/500
77/77 [==============================] - 0s 1ms/sample - loss: 2619870.2403
Epoch 372/500
77/77 [==============================] - 0s 1ms/sample - loss: 2579506.3734
Epoch 373/500
77/77 [==============================] - 0s 1ms/sample - loss: 2631607.8636
Epoch 374/500
77/77 [==============================] - 0s 1ms/sample - loss: 2694962.0779
Epoch 375/500
77/77 [==============================] - 0s 1ms/sample - loss: 2548079.1464
Epoch 376/500
77/77 [===

In [44]:
X_test_d, y_test_d = split_sequence(deaths[25:48], 4)
X_test_d = X_test_d.reshape((X_test_d.shape[0], n_seq, 1, n_steps, n_features))
res_d = model.predict(X_test_d)
mean_squared_error(y_test_d, res_d)

16.106315840520487